In [184]:
from enum import Enum
from itertools import product

import numpy as np
import pandas as pd
import pulp as P

class Task(Enum):
    VAGTHAVENDE_ELEV = 0
    ORDONNANS = 1
    UDKIG = 2
    BJÆRGEMÆRS = 3
    RORGÆNGER = 4
    UDSÆTNINGSGAST = 5
    HU = 6
    PEJLEGAST_A = 7
    PEJLEGAST_B = 8
    DÆKSELEV_I_KABYS = 9

N_GASTS = 60
N_TEAMS = 3
N_SHIFTS = 6
N_TASKS = 10
MAX_SICK = 2
TEAM_SIZE = 20
TEAM_ORDER = [0,1,2]

def team_minmax(team):
    return team*20, team*20+20

## Create synthetic data

In [182]:
dates = pd.date_range(start='3/1/2021', end='3/14/2021').date

history = []
np.random.seed(42)

for dt in dates:
    for shift in range(N_SHIFTS):
        n_sick = np.random.randint(0,MAX_SICK)
        sick = np.random.choice(N_GASTS, n_sick)
        team = TEAM_ORDER[shift % N_TEAMS]
        min_gast, max_gast = team_minmax(team)
        active_gasts = [gast for gast in range(min_gast, max_gast) if gast not in sick]
        for task_no, gast_no in enumerate(np.random.permutation(active_gasts)[:N_TASKS]):
            task_name = Task(task_no).name
            entry = {'date': dt, 'shift': shift, 'gast_no': gast_no, 'task_no': task_no, 'task_name': task_name}
            history.append(entry)
        for gast_no in sick:
            entry = {'date': dt, 'shift': shift, 'gast_no': gast_no, 'task_no': -1, 'task_name': 'SICK'}
            history.append(entry)
df = pd.DataFrame(history)
df.to_csv('../data/synthetic.csv', index=False)
df

,date,shift,gast_no,task_no,task_name
0,2021-03-01,0,0,0,VAGTHAVENDE_ELEV
1,2021-03-01,0,17,1,ORDONNANS
2,2021-03-01,0,15,2,UDKIG
3,2021-03-01,0,1,3,BJÆRGEMÆRS
4,2021-03-01,0,8,4,RORGÆNGER
...,...,...,...,...,...
871,2021-03-14,5,46,6,HU
872,2021-03-14,5,43,7,PEJLEGAST_A
873,2021-03-14,5,53,8,PEJLEGAST_B
874,2021-03-14,5,57,9,DÆKSELEV_I_KABYS


## Solve day

In [253]:
def solve(df, shift, ignore_tasks=[], sick=[]):
    """
    TODO: implement ignore_tasks, to allow for solving the problem without certain tas
    """
    team = TEAM_ORDER[shift % N_TEAMS]
    
    gast_min, gast_max = team_minmax(team)
    gast_range = np.arange(gast_min, gast_max)
    active = [(gast_no not in sick) for gast_no in gast_range]
    stats = df[
        (df.gast_no.isin(gast_range)) 
        & (df.task_no >= 0)
    ].groupby(['gast_no', 'task_no']).size()

    # constants
    coef = np.zeros((TEAM_SIZE, N_TASKS)).astype(int)

    for gast_no, task_no in stats.index:
        norm_gast_no = gast_no - team*TEAM_SIZE
        coef[norm_gast_no , task_no] = stats[gast_no, task_no]
    
    X = P.LpVariable.dicts('x', (range(TEAM_SIZE), range(N_TASKS)), 0, 1, P.LpBinary)

    prob = P.LpProblem('Schedule', P.LpMinimize)
    
    # objective function
    prob += P.lpSum([coef[i][j] * X[i][j] for i in range(TEAM_SIZE) for j in range(N_TASKS)])
        
    # at most one task per gast
    for i in np.arange(TEAM_SIZE)[active]:
        prob += P.lpSum([X[i][j] for j in range(N_TASKS)]) <= 1
        
    # at most one gast per task
    for j in range(N_TASKS):
        prob += P.lpSum([X[i][j] for i in range(TEAM_SIZE)]) == 1
        
    # no tasks for sick gasts
    pass  # TODO
        
    sol = prob.solve()
    for i in range(TEAM_SIZE):
        for j in range(N_TASKS):
            x = X[i][j]
            if x.varValue == 1:
                gast_no = gast_range[i]
                task_name = Task(j)
                yield (gast_no, task_name)
    

for shift in range(N_SHIFTS):
    print(f'Shift {shift}:')
    for gast_no, task_name in solve(df, shift):
        print(f'- gast #{gast_no} does {task_name}')


Shift 0:
- gast #0 does Task.ORDONNANS
- gast #1 does Task.VAGTHAVENDE_ELEV
- gast #2 does Task.HU
- gast #4 does Task.RORGÆNGER
- gast #7 does Task.DÆKSELEV_I_KABYS
- gast #10 does Task.UDKIG
- gast #11 does Task.BJÆRGEMÆRS
- gast #14 does Task.UDSÆTNINGSGAST
- gast #18 does Task.PEJLEGAST_A
- gast #19 does Task.PEJLEGAST_B
Shift 1:
- gast #21 does Task.HU
- gast #22 does Task.ORDONNANS
- gast #24 does Task.UDKIG
- gast #26 does Task.BJÆRGEMÆRS
- gast #29 does Task.VAGTHAVENDE_ELEV
- gast #32 does Task.PEJLEGAST_B
- gast #35 does Task.UDSÆTNINGSGAST
- gast #36 does Task.DÆKSELEV_I_KABYS
- gast #37 does Task.RORGÆNGER
- gast #38 does Task.PEJLEGAST_A
Shift 2:
- gast #41 does Task.VAGTHAVENDE_ELEV
- gast #42 does Task.PEJLEGAST_B
- gast #43 does Task.DÆKSELEV_I_KABYS
- gast #46 does Task.ORDONNANS
- gast #49 does Task.HU
- gast #50 does Task.BJÆRGEMÆRS
- gast #51 does Task.UDKIG
- gast #55 does Task.UDSÆTNINGSGAST
- gast #57 does Task.RORGÆNGER
- gast #58 does Task.PEJLEGAST_A
Shift 3:
